In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import tqdm
import random
import glob
import json

In [2]:
RANDOM_SEED=100 # different random seed for all-negative so that we pick diff attribute pairs.
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [3]:
ZAPPOS_ROOT = '/home/amrith/zappos-50k/'
ZAPPOS_IMAGES_ROOT = '/home/amrith/zappos-50k/ut-zap50k-images-square'

In [4]:
df = pd.read_csv(f'{ZAPPOS_ROOT}/ut-zap50k-data/meta-data-bin.csv')
df['HeelHeight.High.heel'] = df['HeelHeight.4in...4.3.4in'] + df['HeelHeight.5in...over']
df['HeelHeight.Short.heel'] = df['HeelHeight.Flat'] + df['HeelHeight.Under.1in'] + df['HeelHeight.1in...1.3.4in']

In [5]:
df.shape

(50025, 154)

In [6]:
print(sorted(df.columns))

['CID', 'Category.Boots', 'Category.Sandals', 'Category.Shoes', 'Category.Slippers', 'Closure.Adjustable', 'Closure.Ankle.Strap', 'Closure.Ankle.Wrap', 'Closure.Belt', 'Closure.Buckle', 'Closure.Bungee', 'Closure.Button.Loop', 'Closure.Elastic.Gore', 'Closure.Hook.and.Loop', 'Closure.Lace.up', 'Closure.Monk.Strap', 'Closure.Pull.on', 'Closure.Sling.Back', 'Closure.Slip.On', 'Closure.Snap', 'Closure.Spat.Strap', 'Closure.T.Strap', 'Closure.Toggle', 'Closure.Zipper', 'Gender.Boys', 'Gender.Girls', 'Gender.Men', 'Gender.Women', 'HeelHeight.1in...1.3.4in', 'HeelHeight.2in...2.3.4in', 'HeelHeight.3in...3.3.4in', 'HeelHeight.4in...4.3.4in', 'HeelHeight.5in...over', 'HeelHeight.Flat', 'HeelHeight.High.heel', 'HeelHeight.Short.heel', 'HeelHeight.Under.1in', 'Insole.EVA', 'Insole.Gel', 'Insole.Hypoallergenic', 'Insole.Latex.Lined', 'Insole.Leather', 'Insole.Memory.Foam', 'Insole.Moisture.Wicking', 'Insole.Orthotic.Friendly', 'Insole.Padded', 'Insole.Polyurethane', 'Insole.Poron', 'Insole.Remova

In [7]:
# Attribute list
ATTR_LIST_1 = ['Category.Shoes', 'Category.Sandals', 'SubCategory.Oxfords', 'SubCategory.Heel', 'SubCategory.Boot', 'SubCategory.Slipper.Flats', 'SubCategory.Flats', 'SubCategory.Slipper.Heels', 'SubCategory.Athletic', 'SubCategory.Knee.High', 'SubCategory.Crib.Shoes', 'SubCategory.Over.the.Knee', 'HeelHeight.High.heel', 'HeelHeight.Short.heel', 'Closure.Pull.on', 'Closure.Ankle.Strap', 'Closure.Zipper', 'Closure.Elastic.Gore', 'Closure.Sling.Back', 'Closure.Toggle', 'Closure.Snap', 'Closure.T.Strap', 'Closure.Spat.Strap', 'Gender.Men', 'Gender.Boys', 'Material.Rubber', 'Material.Wool', 'Material.Silk', 'Material.Aluminum', 'Material.Plastic', 'ToeStyle.Capped Toe', 'ToeStyle.Square Toe', 'ToeStyle.Snub Toe', 'ToeStyle.Bicycle Toe', 'ToeStyle.Open Toe', 'ToeStyle.Pointed Toe', 'ToeStyle.Almond', 'ToeStyle.Apron Toe', 'ToeStyle.Snip Toe', 'ToeStyle.Medallion']
ATTR_LIST_2 = ['Category.Boots', 'Category.Slippers', 'SubCategory.Mid.Calf', 'SubCategory.Ankle', 'SubCategory.Loafers', 'SubCategory.Boat.Shoes', 'SubCategory.Clogs.and.Mules', 'SubCategory.Sneakers.and.Athletic.Shoes', 'SubCategory.Heels', 'SubCategory.Prewalker', 'SubCategory.Prewalker.Boots', 'SubCategory.Firstwalker', 'Closure.Lace.up', 'Closure.Buckle', 'Closure.Hook.and.Loop', 'Closure.Slip.On', 'Closure.Ankle.Wrap', 'Closure.Bungee', 'Closure.Adjustable',  'Closure.Button.Loop', 'Closure.Monk.Strap', 'Closure.Belt', 'Gender.Women', 'Gender.Girls', 'Material.Suede', 'Material.Snakeskin', 'Material.Corduroy', 'Material.Horse.Hair', 'Material.Stingray', 'ToeStyle.Round Toe', 'ToeStyle.Closed Toe', 'ToeStyle.Moc Toe', 'ToeStyle.Wingtip', 'ToeStyle.Center Seam', 'ToeStyle.Algonquin', 'ToeStyle.Bump Toe', 'ToeStyle.Wide Toe Box', 'ToeStyle.Peep Toe']
FULL_ATTR_LIST = sorted(list(set(ATTR_LIST_1 + ATTR_LIST_2)))
for x in FULL_ATTR_LIST:
    if x not in df.columns: 
        print(x)
print(len(FULL_ATTR_LIST))

78


In [8]:
ATTR_LIST = []
MIN_IMAGES_PER_ATTR = 1000
for a, x in zip(FULL_ATTR_LIST, df.loc[:, FULL_ATTR_LIST].apply(sum, axis=0)):
    if x > MIN_IMAGES_PER_ATTR:
        ATTR_LIST.append(a)
print(len(ATTR_LIST))

38


In [9]:
ATTR_LIST

['Category.Boots',
 'Category.Sandals',
 'Category.Shoes',
 'Category.Slippers',
 'Closure.Ankle.Strap',
 'Closure.Buckle',
 'Closure.Elastic.Gore',
 'Closure.Hook.and.Loop',
 'Closure.Lace.up',
 'Closure.Pull.on',
 'Closure.Slip.On',
 'Closure.Zipper',
 'Gender.Boys',
 'Gender.Girls',
 'Gender.Men',
 'Gender.Women',
 'HeelHeight.High.heel',
 'HeelHeight.Short.heel',
 'Material.Rubber',
 'Material.Suede',
 'SubCategory.Ankle',
 'SubCategory.Clogs.and.Mules',
 'SubCategory.Flats',
 'SubCategory.Heels',
 'SubCategory.Knee.High',
 'SubCategory.Loafers',
 'SubCategory.Mid.Calf',
 'SubCategory.Oxfords',
 'SubCategory.Slipper.Flats',
 'SubCategory.Sneakers.and.Athletic.Shoes',
 'ToeStyle.Almond',
 'ToeStyle.Capped Toe',
 'ToeStyle.Closed Toe',
 'ToeStyle.Moc Toe',
 'ToeStyle.Open Toe',
 'ToeStyle.Peep Toe',
 'ToeStyle.Pointed Toe',
 'ToeStyle.Round Toe']

1. logic to sample attribute list
2. logic to check if attributes are from same category
3. logic to sample images that are positive for the attribute pair

In [10]:
def is_same_category(attrs):
    return len(set([attr.lower().split(".")[0] for attr in attrs])) < len(attrs)
    
def sample_attrs(attributes_list, n_attrs=2):
    return np.random.choice(attributes_list, n_attrs, replace=False)

def get_all_positive_images(df, attrs):
    return df[attrs].apply(sum, axis=1) == len(attrs)

def get_all_negative_images(df, attrs):
    return (1-df[attrs]).apply(sum, axis=1) == len(attrs)

def sample_images(df, selected_images, n_samples):
    return list(np.random.choice(df[selected_images]["CID"].values, n_samples, replace=False))

In [11]:
N_SAMPLES = 8000
POSITIVE_IMAGES_THRESHOLD = 20
NEGATIVE_IMAGES_THRESHOLD = 20
N_SUPPORT = 10
N_QUERY = 10

In [12]:
CID_to_impath = {x.split("/")[-1][:-4].replace(".", "-"):x for x in glob.glob(f"{ZAPPOS_IMAGES_ROOT}/**/*.jpg", recursive=True)}

In [13]:
# main logic for sampling
n_selected = 0
dataset = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
counter = 0
import time
st = time.time()
while n_selected < N_SAMPLES:
    counter += 1
    # get attributes
    attrs = sample_attrs(ATTR_LIST)
    print(attrs)
    # if any pair of the sampled attributes are from the same category then reject
    if is_same_category(attrs):
        # print("Dropped -- same category", attrs)
        continue
    # get images which satisfy the attributes
    positive_images = get_all_positive_images(df, attrs)
    negative_images = positive_images.apply(lambda x: not x)
    assert all([(x != y) for x,y in zip(positive_images, negative_images)])
    # if we want to enforce 0s for all attributes then we need to change the logic above
    # to include get_all_negative_images(df, attrs)
    
    # if the no of images that are positive for the chosen attributes is too less, then reject 
    if (sum(positive_images) < POSITIVE_IMAGES_THRESHOLD) or (sum(negative_images) < NEGATIVE_IMAGES_THRESHOLD) :
        # print("Dropped -- didnt meet threshold", attrs)
        continue
    
    positive = [CID_to_impath[z] for z in sample_images(df, positive_images, N_SUPPORT+N_QUERY)]
    negative = [CID_to_impath[z] for z in sample_images(df, negative_images, N_SUPPORT+N_QUERY)]
    
    dataset[n_selected][0]['support'] = negative[:N_SUPPORT]
    dataset[n_selected][0]['query'] = negative[N_SUPPORT:]
    
    dataset[n_selected][1]['support'] = positive[:N_SUPPORT]
    dataset[n_selected][1]['query'] = positive[N_SUPPORT:]
    
    dataset[n_selected][0]['attributes'] = "!(" + ";".join([a for a in attrs]) + ")"
    dataset[n_selected][1]['attributes'] = ";".join(attrs)
    
    
    n_selected += 1
    print(f'Selected {n_selected}/{counter} Time Elapsed {time.time()-st}')
        

['Category.Boots' 'Material.Rubber']
Selected 1/1 Time Elapsed 0.6934249401092529
['Gender.Girls' 'ToeStyle.Moc Toe']
Selected 2/2 Time Elapsed 1.3782639503479004
['ToeStyle.Round Toe' 'HeelHeight.High.heel']
Selected 3/3 Time Elapsed 2.1024396419525146
['ToeStyle.Open Toe' 'ToeStyle.Moc Toe']
['HeelHeight.Short.heel' 'Gender.Men']
Selected 4/5 Time Elapsed 2.8073413372039795
['Gender.Girls' 'Category.Slippers']
Selected 5/6 Time Elapsed 3.512463331222534
['Gender.Men' 'Closure.Ankle.Strap']
['Category.Sandals' 'Gender.Women']
Selected 6/8 Time Elapsed 4.902745723724365
['ToeStyle.Closed Toe' 'Closure.Elastic.Gore']
Selected 7/9 Time Elapsed 5.608620882034302
['Closure.Slip.On' 'SubCategory.Knee.High']
['Closure.Pull.on' 'Category.Boots']
Selected 8/11 Time Elapsed 6.952747583389282
['SubCategory.Loafers' 'Gender.Women']
Selected 9/12 Time Elapsed 7.690732955932617
['Closure.Pull.on' 'Closure.Slip.On']
['HeelHeight.High.heel' 'Closure.Elastic.Gore']
Selected 10/14 Time Elapsed 8.413410

['ToeStyle.Open Toe' 'ToeStyle.Capped Toe']
['Closure.Pull.on' 'Closure.Buckle']
['Closure.Hook.and.Loop' 'SubCategory.Loafers']
Selected 79/112 Time Elapsed 67.11333918571472
['SubCategory.Loafers' 'Material.Suede']
Selected 80/113 Time Elapsed 67.79780077934265
['Gender.Boys' 'Gender.Girls']
['SubCategory.Loafers' 'ToeStyle.Moc Toe']
Selected 81/115 Time Elapsed 68.49894452095032
['HeelHeight.High.heel' 'SubCategory.Mid.Calf']
Selected 82/116 Time Elapsed 69.19366073608398
['SubCategory.Loafers' 'SubCategory.Knee.High']
['Gender.Boys' 'Closure.Pull.on']
Selected 83/118 Time Elapsed 69.89328908920288
['ToeStyle.Round Toe' 'Gender.Girls']
Selected 84/119 Time Elapsed 70.59667611122131
['SubCategory.Loafers' 'SubCategory.Slipper.Flats']
['Closure.Ankle.Strap' 'ToeStyle.Capped Toe']
['HeelHeight.High.heel' 'Gender.Men']
['Category.Slippers' 'ToeStyle.Pointed Toe']
['SubCategory.Mid.Calf' 'SubCategory.Oxfords']
['SubCategory.Flats' 'ToeStyle.Pointed Toe']
Selected 85/125 Time Elapsed 73.2

Selected 150/223 Time Elapsed 132.1551308631897
['Material.Rubber' 'Closure.Zipper']
Selected 151/224 Time Elapsed 132.85627245903015
['Closure.Buckle' 'SubCategory.Ankle']
Selected 152/225 Time Elapsed 133.57405614852905
['Material.Rubber' 'ToeStyle.Pointed Toe']
Selected 153/226 Time Elapsed 134.27424025535583
['Closure.Pull.on' 'SubCategory.Knee.High']
Selected 154/227 Time Elapsed 135.01358485221863
['ToeStyle.Capped Toe' 'Closure.Slip.On']
Selected 155/228 Time Elapsed 135.72325944900513
['ToeStyle.Capped Toe' 'Closure.Lace.up']
Selected 156/229 Time Elapsed 136.42236733436584
['Closure.Pull.on' 'HeelHeight.Short.heel']
Selected 157/230 Time Elapsed 137.13470602035522
['Closure.Zipper' 'ToeStyle.Closed Toe']
Selected 158/231 Time Elapsed 137.83230352401733
['Closure.Buckle' 'Category.Slippers']
['SubCategory.Sneakers.and.Athletic.Shoes' 'Gender.Girls']
Selected 159/233 Time Elapsed 139.1790337562561
['SubCategory.Clogs.and.Mules' 'Closure.Pull.on']
['ToeStyle.Closed Toe' 'ToeStyle

Selected 227/330 Time Elapsed 199.54732751846313
['SubCategory.Flats' 'ToeStyle.Moc Toe']
Selected 228/331 Time Elapsed 200.25424528121948
['HeelHeight.Short.heel' 'Closure.Elastic.Gore']
Selected 229/332 Time Elapsed 200.94379234313965
['SubCategory.Knee.High' 'Category.Shoes']
['Category.Slippers' 'Closure.Slip.On']
Selected 230/334 Time Elapsed 202.29122114181519
['Closure.Slip.On' 'SubCategory.Oxfords']
Selected 231/335 Time Elapsed 202.97794842720032
['SubCategory.Slipper.Flats' 'Gender.Men']
Selected 232/336 Time Elapsed 203.68401074409485
['Gender.Girls' 'ToeStyle.Pointed Toe']
Selected 233/337 Time Elapsed 204.36819553375244
['Closure.Elastic.Gore' 'Material.Rubber']
Selected 234/338 Time Elapsed 205.05927348136902
['Gender.Women' 'Closure.Slip.On']
Selected 235/339 Time Elapsed 205.76712918281555
['ToeStyle.Capped Toe' 'HeelHeight.Short.heel']
Selected 236/340 Time Elapsed 206.44806003570557
['Category.Sandals' 'Closure.Hook.and.Loop']
Selected 237/341 Time Elapsed 207.1579627

Selected 302/437 Time Elapsed 262.9797172546387
['Gender.Boys' 'SubCategory.Heels']
['Closure.Buckle' 'ToeStyle.Closed Toe']
Selected 303/439 Time Elapsed 264.32021713256836
['Closure.Lace.up' 'Gender.Boys']
Selected 304/440 Time Elapsed 265.0336606502533
['Material.Suede' 'SubCategory.Flats']
Selected 305/441 Time Elapsed 265.73043179512024
['ToeStyle.Almond' 'Closure.Ankle.Strap']
Selected 306/442 Time Elapsed 266.437952041626
['Gender.Women' 'ToeStyle.Closed Toe']
Selected 307/443 Time Elapsed 267.14022064208984
['Gender.Women' 'SubCategory.Flats']
Selected 308/444 Time Elapsed 267.83090376853943
['Gender.Girls' 'HeelHeight.High.heel']
['Category.Boots' 'Closure.Elastic.Gore']
Selected 309/446 Time Elapsed 269.18125224113464
['SubCategory.Slipper.Flats' 'HeelHeight.Short.heel']
Selected 310/447 Time Elapsed 269.91973996162415
['Gender.Men' 'Closure.Lace.up']
Selected 311/448 Time Elapsed 270.6231622695923
['Closure.Hook.and.Loop' 'Closure.Elastic.Gore']
['Closure.Lace.up' 'ToeStyle.

...

Selected 7709/11820 Time Elapsed 6944.918079853058
['Closure.Buckle' 'ToeStyle.Round Toe']
Selected 7710/11821 Time Elapsed 6945.602600812912
['Closure.Pull.on' 'SubCategory.Sneakers.and.Athletic.Shoes']
['SubCategory.Sneakers.and.Athletic.Shoes' 'Gender.Women']
Selected 7711/11823 Time Elapsed 6946.94597530365
['Category.Sandals' 'Gender.Men']
Selected 7712/11824 Time Elapsed 6947.62350487709
['ToeStyle.Open Toe' 'Closure.Elastic.Gore']
Selected 7713/11825 Time Elapsed 6948.318985700607
['SubCategory.Mid.Calf' 'ToeStyle.Capped Toe']
Selected 7714/11826 Time Elapsed 6949.010707616806
['ToeStyle.Almond' 'Closure.Hook.and.Loop']
Selected 7715/11827 Time Elapsed 6949.688703775406
['ToeStyle.Capped Toe' 'Gender.Girls']
Selected 7716/11828 Time Elapsed 6950.3737506866455
['SubCategory.Heels' 'Gender.Women']
Selected 7717/11829 Time Elapsed 6951.055732488632
['Closure.Pull.on' 'SubCategory.Sneakers.and.Athletic.Shoes']
['ToeStyle.Peep Toe' 'Closure.Lace.up']
['Closure.Hook.and.Loop' 'Materia

['ToeStyle.Capped Toe' 'ToeStyle.Closed Toe']
['ToeStyle.Pointed Toe' 'ToeStyle.Closed Toe']
['Material.Rubber' 'SubCategory.Loafers']
Selected 7786/11924 Time Elapsed 7007.712980508804
['SubCategory.Slipper.Flats' 'SubCategory.Flats']
['Category.Slippers' 'ToeStyle.Almond']
['Gender.Women' 'Category.Boots']
Selected 7787/11927 Time Elapsed 7009.032895326614
['Category.Slippers' 'Category.Sandals']
['Gender.Women' 'ToeStyle.Closed Toe']
Selected 7788/11929 Time Elapsed 7009.7451202869415
['SubCategory.Ankle' 'ToeStyle.Peep Toe']
Selected 7789/11930 Time Elapsed 7010.424874305725
['Category.Shoes' 'Gender.Girls']
Selected 7790/11931 Time Elapsed 7011.116827249527
['SubCategory.Ankle' 'SubCategory.Loafers']
['SubCategory.Heels' 'Category.Slippers']
['Material.Rubber' 'Category.Shoes']
Selected 7791/11934 Time Elapsed 7012.445747375488
['Gender.Boys' 'HeelHeight.Short.heel']
Selected 7792/11935 Time Elapsed 7013.117609739304
['Closure.Elastic.Gore' 'Gender.Women']
Selected 7793/11936 Time

Selected 7857/12028 Time Elapsed 7068.522727966309
['Closure.Pull.on' 'ToeStyle.Moc Toe']
Selected 7858/12029 Time Elapsed 7069.222088336945
['Category.Shoes' 'SubCategory.Sneakers.and.Athletic.Shoes']
Selected 7859/12030 Time Elapsed 7070.016346693039
['ToeStyle.Peep Toe' 'SubCategory.Loafers']
['Closure.Lace.up' 'ToeStyle.Peep Toe']
['Closure.Elastic.Gore' 'SubCategory.Slipper.Flats']
Selected 7860/12033 Time Elapsed 7072.022201061249
['SubCategory.Knee.High' 'Gender.Boys']
Selected 7861/12034 Time Elapsed 7072.731675863266
['SubCategory.Sneakers.and.Athletic.Shoes' 'SubCategory.Ankle']
['ToeStyle.Moc Toe' 'SubCategory.Slipper.Flats']
Selected 7862/12036 Time Elapsed 7073.416132688522
['Closure.Lace.up' 'SubCategory.Heels']
Selected 7863/12037 Time Elapsed 7074.1437957286835
['Category.Boots' 'SubCategory.Oxfords']
['HeelHeight.High.heel' 'SubCategory.Clogs.and.Mules']
Selected 7864/12039 Time Elapsed 7075.493699789047
['Closure.Ankle.Strap' 'SubCategory.Knee.High']
['Closure.Hook.an

Selected 7924/12139 Time Elapsed 7132.980217218399
['SubCategory.Heels' 'SubCategory.Loafers']
['SubCategory.Knee.High' 'ToeStyle.Closed Toe']
Selected 7925/12141 Time Elapsed 7133.691962480545
['ToeStyle.Round Toe' 'Closure.Buckle']
Selected 7926/12142 Time Elapsed 7134.372137069702
['SubCategory.Knee.High' 'SubCategory.Clogs.and.Mules']
['HeelHeight.Short.heel' 'ToeStyle.Closed Toe']
Selected 7927/12144 Time Elapsed 7135.054048061371
['Gender.Men' 'Material.Suede']
Selected 7928/12145 Time Elapsed 7135.735080480576
['Closure.Slip.On' 'Closure.Lace.up']
['ToeStyle.Open Toe' 'Category.Boots']
Selected 7929/12147 Time Elapsed 7136.442199707031
['SubCategory.Loafers' 'ToeStyle.Almond']
Selected 7930/12148 Time Elapsed 7137.141718626022
['ToeStyle.Almond' 'Material.Rubber']
Selected 7931/12149 Time Elapsed 7137.837908267975
['SubCategory.Loafers' 'ToeStyle.Almond']
Selected 7932/12150 Time Elapsed 7138.519948005676
['ToeStyle.Closed Toe' 'SubCategory.Sneakers.and.Athletic.Shoes']
Selected

Selected 7997/12246 Time Elapsed 7193.117569684982
['ToeStyle.Round Toe' 'Material.Suede']
Selected 7998/12247 Time Elapsed 7193.795162200928
['HeelHeight.High.heel' 'ToeStyle.Closed Toe']
Selected 7999/12248 Time Elapsed 7194.474837779999
['Gender.Girls' 'Closure.Lace.up']
Selected 8000/12249 Time Elapsed 7195.159035682678


In [14]:
len(dataset)

8000

In [15]:
json.dump(dataset, open(f'zappos-minImg{MIN_IMAGES_PER_ATTR}-alltrue_vs_anyfalse-nsamp{N_SAMPLES}-ns{N_SUPPORT}-nq{N_QUERY}.json', 'w'))